In [318]:
import tensorflow as tf
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [319]:
%%bash
ls

__init__.py
foil2.ipynb


In [327]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [413]:
num_entities = 2
num_relations = 3
emb_dim = 4
ent_embeddings = tf.Variable(tf.random_normal([num_entities, emb_dim],stddev=0.1), tf.float32)
rel_embeddings = tf.Variable(tf.random_normal([num_relations, emb_dim, emb_dim],stddev=0.1), tf.float32)

bool_ent_embeddings = tf.sigmoid(ent_embeddings)
parameters = [ent_embeddings,rel_embeddings]

def scores(rel_ids, arg1_ids, arg2_ids):
    batch_rel_embeddings = tf.gather(rel_embeddings, rel_ids) # [num_facts, emb_dim, emb_dim]
    arg1_embeddings = tf.gather(bool_ent_embeddings, arg1_ids)
    arg2_embeddings = tf.gather(bool_ent_embeddings, arg2_ids) # [num_facts, 1, emb_dim]
    return scores_for_embeddings(batch_rel_embeddings, arg1_embeddings, arg2_embeddings)

def scores_for_embeddings(batch_rel_embeddings, arg1_embeddings,arg2_embeddings):
    expanded_arg1_embeddings = tf.expand_dims(arg1_embeddings, 2)
    expanded_arg2_embeddings = tf.expand_dims(arg2_embeddings, 1)
    result = tf.reduce_sum(expanded_arg1_embeddings * batch_rel_embeddings * expanded_arg2_embeddings, [1,2])
    return result

def log_loss(scores, targets):
    return tf.nn.sigmoid_cross_entropy_with_logits(scores, targets)

num_order_2_rules = 2
use_linear = True
if use_linear:
    rule_violators_arg1_raw = tf.Variable(tf.random_uniform([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg2_raw = tf.Variable(tf.random_uniform([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg1 = tf.maximum(0., tf.minimum(1., rule_violators_arg1_raw))
    rule_violators_arg2 = tf.maximum(0., tf.minimum(1., rule_violators_arg2_raw))
else:
    rule_violators_arg1_raw = tf.Variable(tf.random_normal([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg2_raw = tf.Variable(tf.random_normal([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg1 = tf.sigmoid(rule_violators_arg1_raw)
    rule_violators_arg2 = tf.sigmoid(rule_violators_arg2_raw)

rule_violators = [rule_violators_arg1_raw, rule_violators_arg2_raw]

def rule_losses_for_embeddings(head_embeddings, body_embeddings, arg1s, arg2s):
    rule_head_scores = scores_for_embeddings(head_embeddings,arg1s, arg2s)
    rule_body_scores = scores_for_embeddings(body_embeddings,arg1s, arg2s)
    losses = tf.maximum(0.0, rule_body_scores - rule_head_scores)
    return losses    

def rule_losses_for_relation_indices(heads, bodies, arg1s, arg2s):
    head_embeddings = tf.gather(rel_embeddings, heads) # [num_rules, emb_dim, emb_dim]
    body_embeddings = tf.gather(rel_embeddings, bodies) # [num_rules, emb_dim, emb_dim]
    return rule_losses_for_embeddings(head_embeddings,body_embeddings, arg1s, arg2s)


def rule_losses(heads, bodies):
    head_embeddings = tf.gather(rel_embeddings, heads) # [num_rules, emb_dim, emb_dim]
    body_embeddings = tf.gather(rel_embeddings, bodies) # [num_rules, emb_dim, emb_dim]
    rule_head_scores = scores_for_embeddings(head_embeddings,rule_violators_arg1, rule_violators_arg2)
    rule_body_scores = scores_for_embeddings(body_embeddings,rule_violators_arg1, rule_violators_arg2)
    losses = tf.maximum(0.0, rule_body_scores - rule_head_scores)
    return losses

In [414]:
sess.run(tf.initialize_all_variables())
# sess.run(log_loss(scores([0,1],[1,2],[2,3]), [1.,0]))

In [323]:
sess.run(rule_losses([0,1],[1,2]))

array([ 0.01249681,  0.        ], dtype=float32)

In [384]:
opt_learn = tf.train.AdamOptimizer(learning_rate=0.01)
opt_find = tf.train.AdamOptimizer(learning_rate=1.0)

rule_violation_loss = rule_losses([1,2],[0,1])
training_loss = log_loss(scores([0],[0],[1]),[1.]) + 100 * rule_violation_loss + \
    0.001 * tf.nn.l2_loss(rel_embeddings) + \
    0.001 * tf.nn.l2_loss(ent_embeddings)

opt_find_violation = opt_find.minimize(-rule_violation_loss,var_list=rule_violators)
opt_find_parameters = opt_learn.minimize(training_loss, var_list=parameters)

In [406]:
sess.run(tf.initialize_all_variables())
num_outer_epochs = 2
num_find_violations_epochs = 10
num_training_epochs = 10
num_print_out = 1
for outer_epoch in range(0,num_outer_epochs):
    print("Estimate parameters")
#     sess.run(tf.initialize_variables(parameters))
    for i in range(0,num_training_epochs):
        current_loss,_ = sess.run([training_loss,opt_find_parameters])
#         if i % (num_training_epochs // num_print_out) == 0:
#             print(current_loss)

    print(sess.run(training_loss))
    print(sess.run(scores([0,1,2],[0,0,0],[1,1,1])))

    print("Find Violations")
    sess.run(tf.initialize_variables(rule_violators))
    for i in range(0,num_find_violations_epochs):
        current_loss,_ = sess.run([rule_violation_loss,opt_find_violation])
#         if i % (num_find_violations_epochs // num_print_out) == 0:
#             print(current_loss)
    print(sess.run(rule_violation_loss))
                
sess.run(scores([0,1,2],[0,0,0],[1,1,1])) 
# sess.run(scores([1],[0],[1])) 

Estimate parameters
[ 0.71793658  0.71793658]
[-0.04860514  0.04639324  0.09288298]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.73711264  0.73711264]
[-0.08547477  0.07404413  0.1611322 ]
Find Violations
[ 0.  0.]


array([-0.08547477,  0.07404413,  0.1611322 ], dtype=float32)

In [420]:
def find_violators(heads, bodies):
    r2 = tf.gather(rel_embeddings, heads) # [num_rules, emb_dim, emb_dim]
    r1 = tf.gather(rel_embeddings, bodies) # [num_rules, emb_dim, emb_dim]

    # first score rows of r1 and r2 according to how much r1 > r2
    # row_scores = tf.reduce_sum(r1 - r2, 2)
    row_scores = tf.reduce_sum(r1 - r2, 2)
    # then choose x_i = 1 if row_scores_i > 0 and 0 otherwise
    x = tf.to_float(tf.greater(row_scores,0.0)) # [num_rules, emb_dim]
    # now calculate cell_scores = x * (r1 - r2)
    cell_scores = tf.reduce_sum(tf.expand_dims(x, 2) * (r1 - r2), 1)
    
    y = tf.to_float(tf.greater(cell_scores,0.0))
    
    # choose y_i = 1 if cell_scores_i >0 and 0 otherwise
    return x,y

def loss_of_violators(heads, bodies):
    x,y = find_violators(heads, bodies)
    return rule_losses_for_relation_indices(heads, bodies, tf.stop_gradient(x), tf.stop_gradient(y))

x,y = find_violators([1,2],[0,1])
sess.run(x)

array([[ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  0.,  1.]], dtype=float32)

In [425]:
sess.run(loss_of_violators([1,2],[0,1]))

array([ 0.73759103,  0.43164906], dtype=float32)

In [427]:
single_loss = log_loss(scores([0],[0],[1]),[1.]) + 10 * loss_of_violators([1,2],[0,1]) + \
    0.001 * tf.nn.l2_loss(rel_embeddings) + \
    0.001 * tf.nn.l2_loss(ent_embeddings)
    
single_train = tf.train.AdamOptimizer(learning_rate=0.1)
single_opt_op = single_train.minimize(single_loss)



In [432]:
sess.run(tf.initialize_all_variables())

for i in range(0, 100):
    loss, _ = sess.run([single_loss, single_opt_op])
    if i % 20 == 0:
        print(loss)
        print(sess.run(scores([0,1,2],[0,0,0],[1,1,1])))



[ 3.71109772  3.78702116]
[ 0.2543166   0.04733372  0.09534913]
[ 11.17756748   3.68827319]
[ 0.15720308  0.13676359  0.12247671]
[ 5.01717186  4.46808052]
[ 0.03257356  0.15591231  0.2995961 ]
[ 2.80950856  2.16938758]
[-0.07318658  0.15549208  0.52132118]
[ 0.73133022  0.73133022]
[-0.14534345  0.14946072  0.73195297]
[ 0.76978981  0.76978981]
[-0.18425477  0.13754161  0.92195451]
[ 5.52839947  0.79130191]
[-0.22892624  0.13255742  1.09010649]
[ 5.01397657  0.81637162]
[-0.25145918  0.13034813  1.23513174]
[ 0.829458  0.829458]
[-0.24662238  0.12425942  1.35472548]
[ 0.82726818  0.82726818]
[-0.21618611  0.11501609  1.44911456]
[ 14.85031319   0.81088042]
[-0.22881128  0.12211845  1.52040505]
[ 0.81836629  0.81836629]
[-0.22434784  0.12718137  1.5712595 ]
[ 0.81640077  0.81640077]
[-0.20430467  0.13062583  1.60442173]
[ 0.8058635  0.8058635]
[-0.16992804  0.13276261  1.62278152]
[ 0.78765845  0.78765845]
[-0.12241959  0.13379945  1.62935448]
[ 0.76279283  0.76279283]
[-0.06302503  0.